In [1]:
from coffea.nanoevents import NanoEventsFactory, NanoAODSchema
from coffea import hist, processor
from pocket_coffea.parameters import defaults
import numpy as np
import awkward as ak
import matplotlib.pyplot as plt
from hist import Hist

import sys
import os

from custom_cut_functions import *
from custom_cuts import *

/t3home/mmalucch/micromamba/envs/pocket-coffea/lib/python3.9/site-packages/coffea/util.py:154: FutureWarning: In coffea version v2023.3.0 (target date: 31 Mar 2023), this will be an error.
(Set coffea.deprecations_as_errors = True to get a stack trace now.)
ImportError: coffea.hist is deprecated
  warnings.warn(message, FutureWarning)


In [2]:


filename ="root://storage01.lcg.cscs.ch:1096//pnfs/lcg.cscs.ch/cms/trivcat//store/mc/Run3Summer22EENanoAODv12/GluGlutoHHto4B_kl-1p00_kt-1p00_c2-0p00_TuneCP5_13p6TeV_powheg-pythia8/NANOAODSIM/Poisson60KeepRAW_130X_mcRun3_2022_realistic_postEE_v6-v2/50000/fc603037-ef65-4bbf-9cef-934ecec40bbe.root"
events = NanoEventsFactory.from_root(filename, schemaclass=NanoAODSchema, entry_stop=100000).events()
print("Events read:", len(events))

Events read: 57232


In [3]:
print(events.fields)
print(events.Electron.fields)
print(events.Electron.pt)

['DeepMETResolutionTune', 'TkMET', 'SV', 'LHEReweightingWeight', 'MET', 'TrigObj', 'LHE', 'boostedTau', 'LowPtElectron', 'BeamSpot', 'GenJet', 'Generator', 'Flag', 'CorrT1METJet', 'genWeight', 'GenVisTau', 'FsrPhoton', 'HLTriggerFinalPath', 'Tau', 'IsoTrack', 'SoftActivityJetHT5', 'SoftActivityJetNjets5', 'SoftActivityJetHT10', 'event', 'L1', 'PuppiMET', 'SoftActivityJetHT', 'FatJet', 'GenDressedLepton', 'LHEPdfWeight', 'L1Reco', 'Photon', 'SoftActivityJetNjets10', 'DeepMETResponseTune', 'genTtbarId', 'L1simulation', 'Rho', 'HLTriggerFirstPath', 'SoftActivityJet', 'Muon', 'run', 'LHEWeight', 'Electron', 'PV', 'GenMET', 'GenProton', 'bunchCrossing', 'Jet', 'HLT', 'SubGenJetAK8', 'ChsMET', 'GenIsolatedPhoton', 'PSWeight', 'GenVtx', 'OtherPV', 'luminosityBlock', 'RawMET', 'SoftActivityJetHT2', 'SubJet', 'LHEScaleWeight', 'LHEPart', 'HTXS', 'SoftActivityJetNjets2', 'Pileup', 'CaloMET', 'GenJetAK8', 'RawPuppiMET', 'GenPart']
['seediEtaOriX', 'convVeto', 'cutBased', 'cutBased_HEEP', 'isPFcan

In [4]:
default_parameters = defaults.get_default_parameters()
defaults.register_configuration_dir("config_dir", "/params")

parameters = defaults.merge_parameters_from_files(    default_parameters,
    f"params/object_preselection.yaml",
    f"params/triggers.yaml",
    # f"{localdir}/params/lepton_scale_factors.yaml",
    # f"{localdir}/params/plotting_style.yaml",
    update=True,
)
# print(parameters["object_preselection"]["Muon"]["pt"])

In [5]:
print(events.LHEPart.pdgId)

# Select b-quarks at LHE level
isOutgoing = events.LHEPart.status == 1
print(isOutgoing)

isB = abs(events.LHEPart.pdgId) == 5
# print only events with at least one b-quark
for     i in range(len(isB)):
    if len(isB[isB][i]) > 0:
        print(i)
        print(events.LHEPart.pdgId[i])
        break
bquarks = events.LHEPart[isB & isOutgoing]

[[21, 21, 25, 25, 21], [21, 21, 25, 25, ... 21, 25, 25, 21], [21, 21, 25, 25, 21]]
[[False, False, True, True, True], [False, ... [False, False, True, True, True]]
20
[5, 21, 25, 25, 5]


In [6]:
e=events[20]
print(e.LHEPart.pdgId)
pdg_gen = e.GenPart.pdgId
print(pdg_gen==5)
# select b-quarks at Gen level
isB = abs(pdg_gen) == 5
# select higgs at Gen level
isH = pdg_gen == 25
print(pdg_gen[isB | isH])

b_higgs = e.GenPart[isB | isH]


[5, 21, 25, 25, 5]
[True, False, False, False, True, False, ... False, False, False, False, False]
[5, 25, 25, 5, 25, 25, 25, 25, 25, 25, 25, ... 5, -5, 5, -5, 5, 5, -5, -5, 5, -5, 5]


In [16]:
isHiggs = b_higgs.pdgId == 25
isHard = b_higgs.hasFlags(["fromHardProcess"])
hasTwoChildren = ak.num(b_higgs.childrenIdxG, axis=1) == 2
# print(events.GenPart.childrenIdxG)
# print(ak.num(events.GenPart.childrenIdxG, axis=2))

print(len(e.GenPart.pdgId), e.GenPart.pdgId)
print(len(e.GenPart.childrenIdxG), e.GenPart.childrenIdxG)
print(len(e.GenPart.children.pdgId), e.GenPart.children.pdgId
      )

print(len(b_higgs), b_higgs.pdgId
      )
print(len(b_higgs.childrenIdxG), b_higgs.childrenIdxG
      )
print(len(b_higgs.children.pdgId), b_higgs.children.pdgId
      )

# print(isHiggs)
# print(isHard)
# print(hasTwoChildren)

102 [5, 21, 25, 25, 5, 25, 25, 25, 25, 25, ... -11, 221, 111, 111, 111, 22, 22, 22, 22]
102 [[1282, 1283, 1284], [], [1285], [1286], [1298, ... [1380], [1381], [], [], [], []]
102 [[25, 25, 5], [], [25], [25], [5], [25], ... 22], [22], [22], [22], [], [], [], []]
31 [5, 25, 25, 5, 25, 25, 25, 25, 25, 25, 25, ... 5, -5, 5, -5, 5, 5, -5, -5, 5, -5, 5]
31 [[1282, 1283, 1284], [1285], [1286], [1298], ... [1323], [], [], [], [], [], []]
31 [[25, 25, 5], [25], [25], [5], [25], [25, ... 111, 111], [5], [], [], [], [], [], []]
